k - anonymity analysis

Q1a)
Randomly generate a dataset (dataframe) with eight columns and 50,000 rows. 
Each column should be a categorical variable (of arbitrary name) with three levels (of arbitrary names) in approximately equal proportions.

In [8]:
##Comment: We will randomly assign a value to each row, for each variable. 
##This simulates sampling from a larger population, so will reflect a realistic use-case.
##(Ie, in reality there will never be a *precisely* even spread of a variable between levels.

##To simplify matters we will use the same numerical labels for each column.

#Importing standard libraries
library(plyr)
library(dplyr)
library(tibble)
library(tidyr)


##Choose variable names
variable_names <- LETTERS[1:8] #equivalent to names(random_set) <- c(""A","B","C","D","E","F","G",H")

#Choose a seed to make our calculations reproductible.
set.seed(1) 

#Create random dataset
df <- data.frame(replicate(8,sample(1:3,50000,rep=TRUE)))
names(df) <- variable_names 

#Add row_id column
df <- add_column(df, row_id = 0, .before = 1)
df$row_id <- seq.int(nrow(df))

In [9]:
print(head(df))

  row_id A B C D E F G H
1      1 1 1 1 1 2 3 3 2
2      2 3 3 2 3 3 1 3 3
3      3 1 1 1 2 1 3 1 1
4      4 2 1 3 2 3 1 1 1
5      5 1 2 3 3 2 3 3 1
6      6 3 2 1 1 3 1 1 1


Q1b)
Verify that the proportions of each value are similar for each of the eight columns.

In [10]:
#Create empty dataframe to store results
proportions <- matrix(vector(), nrow = 0, ncol = 2) 

#Loop through each variable, appending to the empty dataframe
for (i in variable_names){
  
    #Append the number of 'A', 'B' and 'C' values for each column.
    proportions <- rbind(proportions, as.matrix( count(df, i) ) ) 
  
}

# Print the proportions of each column and variable (the first column being the top 3 rows,
# the second column the next three rows, etc).
print(proportions)

      A  freq
 [1,] 1 16651
 [2,] 2 16566
 [3,] 3 16783
 [4,] 1 16737
 [5,] 2 16675
 [6,] 3 16588
 [7,] 1 16877
 [8,] 2 16553
 [9,] 3 16570
[10,] 1 16754
[11,] 2 16585
[12,] 3 16661
[13,] 1 16779
[14,] 2 16566
[15,] 3 16655
[16,] 1 16646
[17,] 2 16632
[18,] 3 16722
[19,] 1 16707
[20,] 2 16514
[21,] 3 16779
[22,] 1 16661
[23,] 2 16679
[24,] 3 16660


In [11]:
# Calculate summary statistics for these proportions.
summary(proportions[,2])

   Min. 1st Qu.  Median    Mean 3rd Qu.    Max. 
  16514   16587   16661   16667   16726   16877 

We have a maximum variation of less than 1%, so the proportions of each values are similar 
for each of the eight columns.

Q1c)
How many unique rows (i.e., permutations of category levels) are possible?

In [12]:
##With 8 columns, each having 3 possibilities, there are 3^8 possibilities.
print(3^8)

[1] 6561


Q1d)
Write some code to produce a table and graph which show the frequencies (numbers of groups) by permutation group sizes (up to group size of 10). That is, how many groups are unique combinations (group size = 1), how many groups are made up of a pair of matching combinations (group size = 2), how many groups are made up three the same, etc?
For example, in the table below left (conveniently ordered) of three columns and eight rows, there is one unique row (one group of 1), four rows in pairs (two groups of 2) and three rows in groups of three (one group of 3). Each of the variables in the table has three levels; a, b and c. The table on the right shows the corresponding frequency table, which you should produce for the data you created in part a).


In [14]:
##Step 1. Create a new column with the concatenation of all variables (ie, in the example above,
## this column would start; aab, aab, aab, bca, ...)

df <- df %>% 
  unite(Concat_Columns,variable_names , sep = "", remove = FALSE)


##Step 2. Group by that column, producing a new result column 'Result'.

temp_df <- df %>% dplyr::group_by(Concat_Columns) %>% dplyr::summarize(Result = n()) %>% arrange(desc(Result)) 


##Step 3. Group by our new 'Result' column, to get the final output.
output <- temp_df %>% dplyr::group_by(Result) %>% dplyr::summarize(Count = n()) %>% arrange(Result) 
rm(temp_df)

output <- output %>% dplyr::filter(Result <= 10)
#Print output table of frequencies (numbers of groups) by permutation group sizes (up to 10).
print(output)

# A tibble: 10 x 2
   Result Count
    <int> <int>
 1      1    25
 2      2    90
 3      3   245
 4      4   444
 5      5   672
 6      6   917
 7      7   949
 8      8   892
 9      9   739
10     10   621


Q1e)

Comment upon the distribution of group sizes in d).

We can see that there are 25 groups which are uniquely identifible.
The count increases up to a group size of 7, before slowly decreasing.
If this was not an ad-hoc analysis, we would repeat this investigation multiple times to verify that these figures remain stable (aka using 'Monte Carlo' simulation techniques).





Q1f)
If your random variables were, in fact, meaningful information on individuals, which
group sizes are of most concern from a privacy perspective.

The obvious issue is group sizes of a single household.
Imagine that the above dataset is a list of publicly available information about individuals in a country,
which is associated with a report on health insurance claims (which contains private information, such as health conditions).
This report would contain a list of individuals with severe health issues, and which of the eight categories they fall into.

An attacker could then use the publicly available information to correctly identify what individual household is being referred to,
and thus who has the condition.

However the issue is not limited to just group sizes of a single household.
Imagine a situation where an attacker can use the publicly available information to narrow down the list of people to 3 people
(equivalent to a group size of 3).
Then the attacker would know there is a 33% chance that any individual in the group has the medical condition, and could take advantage
of that. For example, a insurance company could raise premiums, or a potential employer could discrimanate again all individuals
in that group of 3.

A real-life example of this would be estimating someones ethnicity from their postcode.

https://www.ons.gov.uk/aboutus/transparencyandgovernance/freedomofinformationfoi/ethnicgroupstatisticsatpostcodelevel

It is also feasible for small groups to be entirely full of people with a medical condition.
Imagine that there is a 30% chance that someone has a medical condition.
In our example, there are 93 groups of 2 households. There is a 9% chance (30% * 30% ) that two people in any group of 2 both have the condition. With 93 groups, it is highly likely that at least one group has both households with the condition (and thus in our list).
This means that an attacker could again identify people with that medical condition with 100% certainity.

Q1g)
Consider the effect of missing data in the dataset you created in Part a). How might
this complicate the production of a frequency table of group sizes in Part d)?

The obvious question here is what is meant by missing data, and how precise does the frequency table have to be?

If the creator of the list not have the data (ie, bank account details or other private information),
or does the individual row not have a valid value for that data (ie, age of spouse for a single person).

If no valid value exists, we could replace with a placeholder value not used elsewhere in the data (ie for
 age of spouse, use '-1'). We should be cautious of privacy effects though, since that would effectively remove one variable/column from the data, making it easily to identify individuals.
 
If the information is unknown, then one option would be to estimate the value (aka 'impute the data').
For example imagine that an individual has a value of 'A' for the first 7 columns, and an unknown eighth column.
To fill in the remaining column we could gather together all people who also had 'A' in the first 7 columns, but had a valid result in the last column.

We could then randomly reselect a replacement value from those people. 
This could be useful for columns containing data which will be heavily dependant on previous columns. For example igf the 7th column is what county a person lives in, and you want a good placeholder for the city.
Alternatively, we could select from all valid values in the data for all people (useful for, say, age).

If estimating the data is not acceptable, we could seperately return the values with missing data as exceptions, and only
produce a table for values with all data. Alternatively we could map missing data to a seperate level, and proceed as normal.
Obviously if exceptions are rare, that could produce privacy issues.


Q1h)
Imagine the code that you wrote for Part d) was to be deployed in an automated system
that our customers could use independently, on potentially large volumes of data.
Describe how you might deploy the code, and what additional considerations you might
have or any changes to the code you might make. Note: it is not necessary to provide
another version of the code used for 1 d).

There are two main issues with deploying this code in an automated system.

Firstly it would have to be generalised for any input.
In particular, variable values would have to be shortened for the concat function to work without
overrunning a valid size of a variable name (or empty variable names causing issues).
I suggest mapping all of the values in each column to unique integers.

There may also be issues with customers trying to put too much data into the automated system. 
For example, a customer testing a file with a million rows and a thousand columns.
A quick fix to this would be limiting the size of the file that this system would accept, or ending the process with an error message if the process takes too long.

As implied in Q1g), they may pass us a file with missing data.
There are several different ways we could deal with missing data as per my answer to Q1g).
We could add a suggested list of methods (imputting random values, imputting the most common value, excluding rows with missing values, or treating a missing value as it's own level).
However my gut feel is that a conversation would have to be had with the end user to understand their use-case.
Similarly there will almost certainly be issues with misspellings in the raw data presented (ie, Birmingham v Birmimgham), which requires understanding of the business case as to whether it's helpful to 'fix' mispellings.


My recommendation for deploying the code would be a cloud computing API, such as one created from Azure's Machine Learning Studio. This removes most of the data engineering associated with setting up and maintaining the service.

--
One specific issue with an extremely large number of columns is the 'concat' column used in the code.
Since it combines the data from all other columns, we might start getting issues with the 'concat' column being of arbitary size. 
A solution to this would be introducing a hash function to map the concat values into a hash table. This would have a small chance of mapping two different values to the same value. However this could only make groups bigger, not smaller.
Hence any small groups in the data would still correspond to real issues.

.